## run this cell before starting spark cluster connection (corrects module error)

In [1]:
# spark.jars.packages com.databricks:spark-avro_2.11:4.0.0
import os
print os.environ['PYSPARK_PYTHON']
os.environ['PYSPARK_PYTHON']=os.environ['LCG_VIEW'] + '/bin/python'
print os.environ['PYSPARK_PYTHON']

/etc/spark/python
/cvmfs/sft.cern.ch/lcg/views/LCG_93/x86_64-slc6-gcc62-opt/bin/python


### Import necessary spark and python stuff

In [2]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import from_json
from pyspark.ml.feature import VectorAssembler

In [3]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', -1)

In [4]:
df_lookup = spark.read.json("/user/sprenner/lookup_final_mai_small.json")

In [5]:
df_lookup.count()

4492385

In [6]:
#df_april = spark.read.json("/user/sprenner/training_official_april_proportion.json")

In [7]:
#df_april.count()

In [8]:
df_lookup.describe().toPandas()

summary                 BYTES                          CHILD_NAME  \
0  count   4492385               4492385                              
1  mean    5.948377180863274E8   None                                 
2  stddev  1.3544548666077678E9  None                                 
3  min     10000                 AOD.05341490._000001.pool.root.1     
4  max     999999862             log.14217468._004112.job.log.tgz.1   

   CHILD_SCOPE  \
0  4492385       
1  None          
2  None          
3  data12_8TeV   
4  mc16_valid    

                                                                                                             join_key_lookup  
0  4492385                                                                                                                    
1  None                                                                                                                       
2  None                                                                                                                       
3  data12_8TeV:data12_8TeV.00200805.physics_MinBias.recon.NTUP_MINBIAS.r4768_tid01304347_00                                   
4  mc16_valid:mc16_valid.361034.Pythia8EvtGen_A2MSTW2008LO_minbias_inelastic_low.merge.HITS.e3581_s3028_s2939_tid10070623_00

In [9]:
encoded = df_lookup

for elem in ['CHILD_NAME','CHILD_SCOPE']:
    encoded = encoded.withColumn(elem + "Hash", hash(encoded[elem]))

In [10]:
encoded.describe().toPandas()

summary                 BYTES                          CHILD_NAME  \
0  count   4492385               4492385                              
1  mean    5.948377180863274E8   None                                 
2  stddev  1.3544548666077676E9  None                                 
3  min     10000                 AOD.05341490._000001.pool.root.1     
4  max     999999862             log.14217468._004112.job.log.tgz.1   

   CHILD_SCOPE  \
0  4492385       
1  None          
2  None          
3  data12_8TeV   
4  mc16_valid    

                                                                                                             join_key_lookup  \
0  4492385                                                                                                                     
1  None                                                                                                                        
2  None                                                                                                                        
3  data12_8TeV:data12_8TeV.00200805.physics_MinBias.recon.NTUP_MINBIAS.r4768_tid01304347_00                                    
4  mc16_valid:mc16_valid.361034.Pythia8EvtGen_A2MSTW2008LO_minbias_inelastic_low.merge.HITS.e3581_s3028_s2939_tid10070623_00   

         CHILD_NAMEHash       CHILD_SCOPEHash  
0  4492385               4492385               
1  908784.0376401399     -2.788333030691027E8  
2  1.2398889794337907E9  8.74519704197362E8    
3  -2147482214           -2057006107           
4  2147482845            2109319881

In [11]:
#df_april = df_april.withColumn("join_key_april", concat_ws(':', df_april["datasetScope"], df_april["dataset"]))
#df_april.createOrReplaceTempView("april")

In [12]:
encoded.write.format("json").save("/user/sprenner/lookup_final_mai_small_encoded.json")